#### Start

In [1]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
# Load token from file
with open('/content/drive/MyDrive/hf_token.txt', 'r') as file:
    hf_token = file.read().strip()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [ ]:
!pip install python-dotenv
!pip install -q -U langchain huggingface_hub sentence_transformers faiss-gpu
!pip install -U langchain-community
!pip install transformers datasets torch scikit-learn numpy

In [4]:
os.chdir('/content/drive/MyDrive/RAGAS')
from config_setup import *
from fn_ragas import *

#### RAG Main

In [ ]:
# Setup directories
setup_directory_structure()
setup_environment()

# Initialize RAG system
rag = RAGSystem(
    documents_path=str(Config.DATA_DIR),
)

# Process documents
rag.load_documents()
rag.create_vector_store()
rag.setup_qa_chain()

# Save vector store
rag.save_vector_store(str(Config.VECTOR_STORE_DIR))

print("RAG system setup complete!")

# Interactive query loop
while True:
    question = input("\nEnter your question (or 'quit' to exit): ")
    if question.lower() == 'quit':
        break

    result = rag.query(question)
    print("\nAnswer:", result['answer'])
    print("\nSources used:")
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"\nSource {i}:")
        print(doc[:200] + "..." if len(doc) > 200 else doc)

#### RAGAS Main

In [ ]:
evaluation_questions = {
    "What are the primary uses of maize in industry?":
        "Maize is used for food, animal feed, and as a raw material in various industries like starch and textile.",
    "What is the optimal time for chickpea planting for  peninsular  India?":
        "For  peninsular  India,  first  fort night of  October  is the  best  time  for  chickpea planting."
}

# Initialize your RAG system
rag = RAGSystem(
    documents_path=str(Config.DATA_DIR),
)
rag.load_documents()
rag.create_vector_store()
rag.setup_qa_chain()

# Initialize evaluator
evaluator = RAGEvaluator(rag_system=rag, evaluation_questions=evaluation_questions)

# Run evaluation and print results
eval_results = evaluator.evaluate()
print(format_evaluation_results(eval_results))